In [1]:
import requests
import sys
import math

In [147]:
#Web Scraping
bigDict = {}

def findNames(https):
    response = requests.get(https, timeout = 2)
    #print(f"response: {response.text}")
    #print(type(response.text))

    sfnIndeces = []
    countryNames = []

    if response.status_code == 200:
        index = response.text.find('class="longFormName"')
        sfnIndeces.append(index)
        while index != -1:
            index = response.text.find('class="longFormName"', index+1)
            if index != -1:
                sfnIndeces.append(index)
        #print(sfnIndeces)
        for index in sfnIndeces:
            delimitor = response.text[index+93:index+200].find("<")
            countryNames.append(response.text[index+93:index+93+delimitor-20])
        return countryNames
    else:
        return []

#Power Index Scraping
powerIndexHTTPS = "https://www.globalfirepower.com/countries-listing.php"
response = requests.get(powerIndexHTTPS)
if len(findNames(powerIndexHTTPS)) == 0:
    print("There was an error in processing your https request (Country Names)")
    sys.exit()

if response.status_code == 200:
    powerIndeces = []
    pi_dict = {}
    index = response.text.find('class="pwrIndxContainer"')
    powerIndeces.append(float(response.text[index+84:index+90]))
    while index != -1:
        index = response.text.find('class="pwrIndxContainer"', index+1)
        if index != -1:
            powerIndeces.append(float(response.text[index+84:index+90]))
    #print(powerIndeces)
    
    pi_dict = dict(zip(findNames(powerIndexHTTPS), powerIndeces))
    #print(pi_dict)
else:
    print("There was an error processing your https request (Power Indeces)")

#Military Budget Scraping
militaryBudgetHTTPS = "https://www.globalfirepower.com/defense-spending-budget.php"
response = requests.get(militaryBudgetHTTPS)
if len(findNames(militaryBudgetHTTPS)) == 0:
    print("There was an error in processing your https request (Names: Power Index)")
    sys.exit()

if response.status_code == 200:
    militaryBudgets = []
    mb_dict = {}
    index = response.text.find('class="valueContainer"')
    delimiter = response.text[index+227: index+300].find(" ")
    number = response.text[index+227:index+227+delimiter]
    while number.find(",") != -1:
        number = number.replace("," , "")
    militaryBudgets.append(float(number))
    while index != -1:
        index = response.text.find('class="valueContainer"', index+1)
        delimiter = response.text[index+227: index+300].find(" ")
        number = response.text[index+227:index+227+delimiter]
        while number.find(",") != -1:
            number = number.replace("," , "")
        if(index != -1):
            militaryBudgets.append(float(number))
    mb_dict = dict(zip(findNames(militaryBudgetHTTPS), militaryBudgets))
    #print(mb_dict)
else:
    print("There was an error processing your https request (Military Budget)")

for key in pi_dict.keys():
    bigDict[key] = [pi_dict[key], mb_dict[key]]
print(bigDict)


#Convert military rankings from negative exponential to positive linear
#outliers = {}
for key in bigDict.keys():
    bigDict[key] = [math.pow(math.e, (bigDict[key][0]-5)*-5)/100000000, bigDict[key][1]/1000000000]
    #if bigDict[key][1] > 200:
        #outliers[key] = bigDict[key]
print(bigDict)

#Remove Outliers
#for key in outliers.keys():
    #del bigDict[key]

#Make find the b value in the linear regression y=mx+b
    #Subtract the b value from both sides to give y-b=mx so that no b value is necessary when calculating the linear regression model
"""sum = 0
for value in bigDict.values():
    sum += value[0]
b = sum/len(bigDict) * -1
#print(b)
for key in bigDict.keys():
    bigDict[key][0] += b
sum = 0
for value in bigDict.values():
    sum += value[0]
print(sum/len(bigDict)) #This number represents the error in the b value for each country
print(bigDict)"""

{'United States': [0.0699, 831781000000.0], 'Russia': [0.0702, 109000000000.0], 'China': [0.0706, 227000000000.0], 'India': [0.1023, 74000000000.0], 'South Korea': [0.1416, 44700000000.0], 'United Kingdom': [0.1443, 62816382000.0], 'Japan': [0.1601, 53000000000.0], 'Turkiye': [0.1697, 40000000000.0], 'Pakistan': [0.1711, 6349876689.0], 'Italy': [0.1863, 31600000000.0], 'France': [0.1878, 49730000000.0], 'Brazil': [0.1944, 24752000000.0], 'Indonesia': [0.2251, 25000000000.0], 'Iran': [0.2269, 9954451000.0], 'Egypt': [0.2283, 9400000000.0], 'Australia': [0.2515, 52559000000.0], 'Israel': [0.2596, 24400000000.0], 'Ukraine': [0.2598, 42000000000.0], 'Germany': [0.2847, 55941410000.0], 'Spain': [0.2882, 22000000000.0], 'Poland': [0.2917, 38360000000.0], 'Vietnam': [0.3158, 7900000000.0], 'Saudi Arabia': [0.3235, 71720000000.0], 'Taiwan': [0.3302, 19100000000.0], 'Thailand': [0.3389, 7700000000.0], 'Algeria': [0.3589, 21600000000.0], 'Canada': [0.3813, 26500000000.0], 'Argentina': [0.3823, 2

'sum = 0\nfor value in bigDict.values():\n    sum += value[0]\nb = sum/len(bigDict) * -1\n#print(b)\nfor key in bigDict.keys():\n    bigDict[key][0] += b\nsum = 0\nfor value in bigDict.values():\n    sum += value[0]\nprint(sum/len(bigDict)) #This number represents the error in the b value for each country\nprint(bigDict)'

In [149]:
# allows me to take the points and plot them in desmos to ensure accuracy
for value in bigDict.values():
    print(f"({value[1]}, {value[0]})")


(831.781, 507.66371805614517)
(109.0, 506.90279331529)
(227.0, 505.89000085871396)
(74.0, 431.73814388504184)
(44.7, 354.7166337868907)
(62.816382, 349.9601378176232)
(53.0, 323.37713930778966)
(40.0, 308.22167744272394)
(6.349876689, 306.07165954250775)
(31.6, 283.6721744325216)
(49.73, 281.55259149582594)
(24.752, 272.4129888244302)
(25.0, 233.64882869240031)
(9.954451, 231.55542368715774)
(9.4, 229.9401956151087)
(52.559, 204.75604703542854)
(24.4, 196.62910845725239)
(42.0, 196.4325776305858)
(55.94141, 173.4378385688673)
(22.0, 170.42907981861578)
(38.36, 167.47251630610427)
(7.9, 148.4605484654362)
(71.72, 142.8534466365051)
(19.1, 138.14712715616588)
(7.7, 132.26655679345726)
(21.6, 119.67972974149829)
(26.5, 106.99897511458146)
(2.544, 106.46531549983553)
(11.349506, 97.01050349042808)
(13.493, 93.2999205265655)
(14.824, 84.971784646386)
(5.974, 81.84434863517501)
(2.69534615, 71.04541042513789)
(4.1, 68.98018269645783)
(2.7, 52.13410056462833)
(3.5, 50.54273702169628)
(6.99504

In [144]:
#Linear Regression Machine Learning Model
minTotalResidual = sys.float_info.max   

#Find the maximum and minimum possible slopes for initial minm and maxm to disinclude exessivly large and small guesses
maxSlope = sys.float_info.min
minSlope = sys.float_info.max
for key in bigDict.keys():
    localM = abs(bigDict[key][0] / (bigDict[key][1]+1))
    if localM > maxSlope:
        maxSlope = localM
    elif localM < minSlope:
        minSlope = localM
print(f"min: {minSlope}, max: {maxSlope}")

while(maxSlope-minSlope > 0.0000000001):
    predictedValuesMax = {}
    predictedValuesMin = {}
    residualsMax = {}
    residualsMin = {}
    m = (maxSlope - minSlope)/2 + minSlope      #This will be our next slope
    localBestResid = 0
    localBestSlope = 0
    maxResid = 0
    minResid = 0
    for key in bigDict:
        predictedValuesMax[key] = maxSlope*bigDict[key][1]
        predictedValuesMin[key] = minSlope*bigDict[key][1]
    #print(predictedValues)
    for key in bigDict:
        residualsMax[key] = bigDict[key][0] - predictedValuesMax[key]
        residualsMin[key] = bigDict[key][0] - predictedValuesMin[key]
    for key in bigDict:
        maxResid += abs(residualsMax[key])
        minResid += abs(residualsMin[key])
    print(f"Max: {maxSlope}")
    print(f"Min {minSlope}")
    if maxResid < minResid:
        localResid = maxResid
        localSlope = maxSlope
        minSlope = m
    else:
        localResid = minResid
        localSlope = minSlope
        maxSlope = m
    
    if localResid < minTotalResidual:
        minTotalResidual = abs(localResid)
        bestSlope = localSlope

print(m)

min: 1.0315501453866134e-09, max: 4164.310130543848
Max: 4164.310130543848
Min 1.0315501453866134e-09
Max: 2082.1550652724395
Min 1.0315501453866134e-09
Max: 1041.0775326367357
Min 1.0315501453866134e-09
Max: 520.5387663188836
Min 1.0315501453866134e-09
Max: 520.5387663188836
Min 260.26938315995756
Max: 520.5387663188836
Min 390.4040747394206
Max: 520.5387663188836
Min 455.4714205291521
Max: 520.5387663188836
Min 488.0050934240179
Max: 504.27192987145077
Min 488.0050934240179
Max: 496.13851164773433
Min 488.0050934240179
Max: 492.07180253587615
Min 488.0050934240179
Max: 490.038447979947
Min 488.0050934240179
Max: 489.02177070198246
Min 488.0050934240179
Max: 488.5134320630002
Min 488.0050934240179
Max: 488.25926274350905
Min 488.0050934240179
Max: 488.1321780837635
Min 488.0050934240179
Max: 488.0686357538907
Min 488.0050934240179
Max: 488.0368645889543
Min 488.0050934240179
Max: 488.0209790064861
Min 488.0050934240179
Max: 488.013036215252
Min 488.0050934240179
Max: 488.0090648196349

In [123]:
def predict(military_spending):
    powerIndex = m*military_spending
    previous = ""
    wentIn = False
    for key in bigDict.keys():
        if powerIndex > bigDict[key][0]:
            if previous == "United States":
                print(f"Your country's power index is {powerIndex}. This is most similar to {previous}'s military")
            elif bigDict[previous][0]-powerIndex < powerIndex -bigDict[key][0]:
                print(f"Your country's power index is {powerIndex}. This is most similar to {previous}'s military")
            else:
                print(f"Your country's power index is {powerIndex}. This is most similar to {key}'s military")
            wentIn = True
            break
        previous = key
    if not wentIn:
        print(f"Your country's power index is {powerIndex}. This is most similar to Iceland's military")

In [138]:
#Predict the power score of a country given its military spending (in billions USD)
predict(0.17)

Your country's power index is 0.12472661726596766. This is most similar to Sri Lanka's military
